# 3-Step Automation using ```pipeline2```

## Imports and plot settings

Run this once

In [5]:
import os
import json
import enum

import numpy as np
from matplotlib import pyplot as plt
import h5py as h5
import specpy as sp

import pipeline2 as p2

# plotting params
%matplotlib inline
# use this to change size of plots
plt.rcParams['figure.figsize'] = [6, 6]

## Code definition for a single acquisition

Run this once

In [4]:
def run_pipeline(save_path, im, hours_to_image, ov_json, sted1_json, sted2_jsons, start_coords, sigma_ov,
                 thresholds_ov, sigma_sted1, threshold_sted1, ov_moves, ov_fovs=None, ov_psz=None, ov_fovs_first=None, 
                 sted1_fovs=None, sted1_pszs=None, sted2_fovs=None, sted2_pszs=None,
                 ov_json_imspector=None, sted1_json_imspector=None, sted2_jsons_imspector=None,
                 onecolor_ov=False, skip_details=False, n_reps_detail=1, sample_name=None,
                 ov_mode=None, sted1_mode=None, sted2_modes=None):
      
    # make output dir if it does not exist already
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # init levels and name handler
    pll = p2.PipelineLevels('field', 'sted1', 'sted2')
    name_handler = p2.DefaultNameHandler(save_path, pll)

    # abuse name_handler to get h5 filename
    # calling with empty index gives just the random hash
    h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

    # open resulting h5 file in context manager
    with h5.File(h5file_name, mode='a') as h5fd:
        data_store = p2.HDF5DataStore(h5fd, pll)

        # init pipeline
        pl = (p2.AcquisitionPipeline('spot-pair-pipeline' if sample_name is None else sample_name)
                .withImspectorConnection(p2.imspector.ImspectorConnection(im).withVerbose())
                .withPipelineLevels(pll)
                .withNameHandler(name_handler)
                .withAddedStoppingCondition(p2.stoppingcriteria.TimedStoppingCriterion(hours_to_image * 60 * 60)))

        # attach h5 data store
        pl.withDataStorage(data_store)

        # overview task generator: move and hold focus
        atg_overview = (p2.taskgeneration.AcquisitionTaskGenerator(pll.field, 
                            p2.taskgeneration.DefaultLocationRemover( # 1. load overview JSON and ignore locations there
                                p2.taskgeneration.JSONFileConfigLoader(
                                    [ov_json],
                                    [ov_json_imspector] if not ov_json_imspector is None else None
                                )
                            ),
                            p2.taskgeneration.DefaultStageOffsetsSettingsGenerator( # 2. (x,y) spiral coordinates from start_pos
                                p2.taskgeneration.SpiralOffsetGenerator().withStart(start_coords).withFOV(ov_moves)
                            ),
                            p2.taskgeneration.DefaultScanModeSettingsGenerator( # 3. set overview mode
                                ['xyz' if ov_mode is None else ov_mode] , True
                            ),
                            p2.taskgeneration.DifferentFirstFOVSettingsGenerator(ov_fovs, ov_psz, ov_fovs_first), # 4. set FOV
                            p2.taskgeneration.DefaultStageOffsetsSettingsGenerator( # 5. change z for focus hold
                                p2.detection.SimpleLegacyFocusHold(
                                    p2.taskgeneration.NewestDataSelector(pl, pll.field)
                                ).withVerbose(True)
                            )
                    )
                    .withDelay(.4)) # some delay in setting parameters (e.g. for stage movement)

        # init pair detector in overview
        if not onecolor_ov:
            detector = p2.detection.LegacySpotPairFinder(
                p2.taskgeneration.NewestDataSelector(pl, pll.field),
            sigma_ov, thresholds_ov).withPlotDetections(True).withVerbose()
        else:
            detector = p2.detection.SimpleSingleChannelSpotDetector(
                p2.taskgeneration.NewestDataSelector(pl, pll.field),
                sigma_ov, thresholds_ov[0] # use channel 0
            ).withPlotDetections(True).withVerbose()

        # detail task generation: settings from last overview + new FOV + detection
        # optionally repeat measurement to check reproducibility
        atg_sted1 = p2.taskgeneration.AcquisitionTaskGenerator(pll.sted1,    
                                 p2.taskgeneration.DefaultLocationKeeper( # 1. take stage pos of overview
                                     p2.taskgeneration.NewestSettingsSelector(pl, pll.field)
                                 ),
                                 p2.taskgeneration.DefaultLocationRemover( # 2. load param (sets) from JSON, ignore loc
                                     p2.taskgeneration.JSONFileConfigLoader(
                                         [sted1_json],
                                         [sted1_json_imspector] if not sted1_json_imspector is None else None,
                                         False)
                                 ),
                                 p2.taskgeneration.DefaultScanModeSettingsGenerator( # 3. set to desired modes (xy, xyz, ...)
                                     ['xyz' if sted1_mode is None else sted1_mode] , True
                                 ),
                                 p2.taskgeneration.DefaultFOVSettingsGenerator(sted1_fovs, sted1_pszs), # 4. set FOVs
                                 p2.taskgeneration.DefaultScanOffsetsSettingsGenerator(
                                     p2.detection.SimpleLocationRepeater( # 5. locations from detector (optionally repeated) 
                                         detector, n=n_reps_detail
                                     )
                                 ))
        atg_detail.withDelay(.2)

        # overview callback: re-add overview
        pl.withCallbackAtLevel(atg_overview, pll.field)
        # overview callback 2: detect & do detail
        pl.withCallbackAtLevel(atg_sted1, pll.field)
        
        
        # HACK: optionally clear all pending sted images after first one has been acquired
        def clear_details(pl):
            dets = [(prio, idx, val) for (prio, idx, val) in pl.queue.queue if prio == pll.sted]
            for (prio, idx, val) in dets:
                pl.queue.queue.remove((prio, idx, val))
        # only do this if user wants to
        if skip_details:
            pl.withCallbackAtLevel(clear_details, pll.sted1)
        
        
        # init pair detector in overview
        if not onecolor_sted1:
            detector_sted = p2.detection.LegacySpotPairFinder(
                p2.taskgeneration.NewestDataSelector(pl, pll.sted1),
            sigma_sted1, thresholds_sted1).withPlotDetections(True).withVerbose()
        else:
            detector_sted = p2.detection.SimpleSingleChannelSpotDetector(
                p2.taskgeneration.NewestDataSelector(pl, pll.sted1),
                sigma_sted1, thresholds_sted1[0] # use channel 0
            ).withPlotDetections(True).withVerbose()
        
        # detail task generation: settings from last overview + new FOV + detection
        # optionally repeat measurement to check reproducibility
        atg_sted2 = p2.taskgeneration.AcquisitionTaskGenerator(pll.sted2,    
                                 p2.taskgeneration.DefaultLocationKeeper( # 1. take stage pos of overview
                                     p2.taskgeneration.NewestSettingsSelector(pl, pll.sted1)
                                 ),
                                 p2.taskgeneration.DefaultLocationRemover( # 2. load param (sets) from JSON, ignore loc
                                     p2.taskgeneration.JSONFileConfigLoader(
                                         sted2_jsons,
                                         sted2_jsons_imspector if not det_jsons_imspector is None else None,
                                         False)
                                 ),
                                 p2.taskgeneration.DefaultScanModeSettingsGenerator( # 3. set to desired modes (xy, xyz, ...)
                                     sted2_modes if not sted2_modes is None else ['xyz'] * len(sted2_jsons),
                                     False),
                                 p2.taskgeneration.DefaultFOVSettingsGenerator(sted2_fovs, sted2_pszs), # 4. set FOVs
                                 p2.taskgeneration.DefaultScanOffsetsSettingsGenerator(# 5. locations from detector 
                                         detector_sted
                                     )
                                 )
        atg_detail.withDelay(.1)
        
        pl.withCallbackAtLevel(atg_sted2, pll.sted1)
        
        # call overview atg once to add first measurement to pipeline
        atg_overview(pl)

        # GO
        pl.run()

## 1) initialize acquisition list and connect to Imspector

Run this to initialize the acquisition queue and connect to imspector
Re-run to clear queued acquisitions

In [ ]:
# init acq. list
acquisitions = []

# imspector object
im = sp.Imspector()

## 2) Enqueue an acquisition

Set the parameters for a new acquisition and add it to the queue.
** Acquisition will be run at the current stage coordinates**

In [ ]:
params = {}

### CHANGE ME!
params['save_path'] = 'C:/Users//RESOLFT/Desktop/auto-pairs/20180821-tetraspeck'


##### (OPTIONALLY) CHANGE US!

# how long to image
params['hours_to_image'] = 12

### PARAMETER FILES
# paths of the parameters files
# we can use multiple for the second SETD measurement (e.g. to do both xz and yz)
params['ov_json'] = 'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180820_ov.json'
params['sted1_json'] = 'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180820_detail.json'
params['sted2_jsons'] = [
    'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180820_detail.json',
    'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180820_detail_3d.json'
]

### IMSPECTOR SETTINGS, optional
# paths to imspector setting files, set to None if you do not want to change settings (e.g. SLM parameters)
params['ov_json_imspector'] = None
params['sted1_json_imspector'] = None
params['sted2_jsons_imspector'] = None
# NB: needs to be of the same format as ov_json and det_jsons
# uncomment lines below for example (same settings for all)
#params['ov_json_imspector'] = 'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default.json'
#params['sted1_json_imspector'] = 'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default.json'
#params['sted2_jsons_imspector'] = [
#    'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default.json',
#    'C:/Users/RESOLFT/Desktop/config_json/old_pipeline/20180821_imspector_settings_default2.json'
#]


### DETECTOR SETTINGS
# spot pair detection parameters (~expected size, thresholds for each channel)
params['sigma_ov'] = 2
params['thresholds_ov'] = [0.9, 0.7]
# whether to detect only in first channel (and not look for pairs)
params['onecolor_ov'] = False

# detection settings in first STED image
params['sigma_sted1'] = 2
params['thresholds_sted1'] = [0.9, 0.7]
params['onecolor_sted1'] = False

### FOV and pixel size of overviews
# NB: we can set a bigger z stack for first stack
# NB: all sizes are in meters!
#params['ov_fovs'] = [[5e-05, 5e-05, 0.5e-5]]
#params['ov_psz'] = [[1.5e-7, 1.5e-7, 2.5e-7]]
#params['ov_fovs_first'] = [[5e-05, 5e-05, 0.5e-5]]
params['ov_moves'] = [6e-5, 6e-5] # how much to move in overview spiral
# (NB: we make it larger than FOV to avoid small overlaps)
#params['sted1_fovs'] = [[3e-06, 3e-06, 1.4e-6]] # STED FOV
#params['sted1_pszs'] = [[2e-8, 2e-8, 2e-7]] # STED Pixelsize
#params['sted2_fovs'] = [[3e-06, 3e-06, 1.4e-6],[3e-06, 3e-06, 1.4e-6]] # STED FOV
#params['sted2_pszs'] = [[2e-8, 2e-8, 2e-7],[3e-06, 3e-06, 1.4e-6]] # STED Pixelsize


### Debug, QC options
params['skip_details'] = False # set to True to only image 1 detail image per field
params['n_reps_detail'] = 1 # how often to repeat each detail image


### SCAN MODES (Optional)
# which scan mode (xy, xyz, ...) to use for overviews and details
# This may be None, in which case we simply use mode set in file
params['ov_mode'] = None
# e.g.
#params['ov_mode'] = 'xyz'
params['sted1_mode'] = None
params['sted2_modes'] = None
# e.g.
#params['sted1_mode'] = 'xyz'
#params['sted2_modes'] = ['xyz', 'xy'] 
# NB: this needs to be alist of the same size as settings for details


### Things that are set automatically

# ensure we use slashes and not backslashes
params['save_path'] = params['save_path'].replace(os.sep, '/')
# sample name, will be set automatically
params['sample_name'] = params['save_path'].strip('/').rsplit('/')[-1]
# start at current coordinates, do not change!
params['start_coords'] = p2.imspector.get_current_stage_coords(im)

# add to queue
acquisitions.append(params)

# print the currently queued acquisitions
print("""
Queued Acquisitions:
====================
""")
for ac in acquisitions:
    print(json.dumps(ac, indent=1))

## 3) Run the acquisitions

Execute cell below to run the enqueued acquisitions

In [ ]:
# go through queued acquisitions (in reverse)
# and run them
for ac in reversed(acquisitions):
    # pass Imspector as well (we excluded it until now because otherwise, the JSON print above would not work)
    ac_t = ac
    ac_t['im'] = im
    run_pipeline(**ac)

# Reset queued acquisitions
# NB: if you cancelled a run, this might not be executed,
#     make sure to clear old acquisitions manually (step 1)
acquisitions = []